In [1]:
import collections

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

tfe = tf.contrib.eager
tfe.enable_eager_execution()

### Basics
There are three important concepts associated with TensorFlow Distributions shapes:
* *Event shape* describes the shape of a single draw from the distribution; it may be dependent across dimensions. For scalar distributions, the event shape is []. For a 5-dimensional MultivariateNormal, the event shape is [5].
* *Batch shape* describes independent, not identically distributed draws, aka 'batch' of distributions.
* *Sample shape* describes independent, identically distributed draws of batches from the distribution family.

The event shape and the batch shape are properties of a `Distribution` object, whereas the sample shape is associated with a specific call to sample or `log_prob`.

### Scalar Distributions
As we noted above, a `Distribution` object has defined event and batch shapes. We'll start with a utility to describe distributions.

In [2]:
def describe_distributions(distributions):
    print('\n'.join([str(d) for d in distributions]))

In this section we'll explore *scalar* distributions: distributions with an event shape of []. A typical example is the Poisson distribution, specified by a rate.

In [3]:
poisson_distributions = [
    tfd.Poisson(rate=1., name='One Poisson Scalar Batch'),
    tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons'),
    tfd.Poisson(rate=[[1., 10., 100.], [2, 20., 200.]],
               name='Two-by-three Poissons'),
    tfd.Poisson(rate=[1.], name='One Poisson Vector Batch'),
    tfd.Poisson(rate=[[1.]], name='One Poisson Expanded Batch')
]

describe_distributions(poisson_distributions)

tfp.distributions.Poisson("One Poisson Scalar Batch/", batch_shape=(), event_shape=(), dtype=float32)
tfp.distributions.Poisson("Three Poissons/", batch_shape=(3,), event_shape=(), dtype=float32)
tfp.distributions.Poisson("Two-by-three Poissons/", batch_shape=(2, 3), event_shape=(), dtype=float32)
tfp.distributions.Poisson("One Poisson Vector Batch/", batch_shape=(1,), event_shape=(), dtype=float32)
tfp.distributions.Poisson("One Poisson Expanded Batch/", batch_shape=(1, 1), event_shape=(), dtype=float32)


The Poisson distribution is a scalar distribution, so its event shape is always []. If we specify more rates, these show up in the batch shape. The final pair of examples is interesting: there is only a single rate, but because that rate is embedded in a numpy array with non-empty shape, that shape becomes the batch shape.

The standard Normal distribution is also a scalar. Its event shape is [], just like for the Poisson, but we'll play with it to see our first example of *broadcasting*. The Normal is specified using `loc` and `scale` parameters.

In [4]:
normal_distribution = [
    tfd.Normal(loc=0., scale=1., name='Standard'),
    tfd.Normal(loc=[0.], scale=1., name='Standard Vector Batch'),
    tfd.Normal(loc=[0., 1, 2, 3], scale=1., name='Different Locs'),
    tfd.Normal(loc=[0., 1, 2, 3], scale=[[1.], [5.]],
              name='Broadcasting Scale')
]

describe_distributions(normal_distribution)

tfp.distributions.Normal("Standard/", batch_shape=(), event_shape=(), dtype=float32)
tfp.distributions.Normal("Standard Vector Batch/", batch_shape=(1,), event_shape=(), dtype=float32)
tfp.distributions.Normal("Different Locs/", batch_shape=(4,), event_shape=(), dtype=float32)
tfp.distributions.Normal("Broadcasting Scale/", batch_shape=(2, 4), event_shape=(), dtype=float32)


The interesting example above is the `Broadcasting Scale` distribution. The `loc` parameter has shape [4], and the `scale` parameter has shape `[2, 1]`. Using Numpy broadcasting rules, the batch shape is `[2, 4]`

#### Sampling Scalar Distributions
There are two main things we can do with distributions: we can `sample` from them and we can compute `log_probs`. Let's explore sampling first. The basic rule is that when we sample from a distribution, the resulting Tensor has shape `[sample_shape, batch_shape, event_shape]`, where `batch_shape` and `event_shape` are provided by the `Distribution` object, and `sample_shape` is provided by the call to `sample`. For scalar distributions, `event_shape = []`, so the Tensor returned from sample will have shape `[sample_shape, batch_shape]`. Let's try it:

In [5]:
def describe_sample_tensor_shape(sample_shape, distribution):
    print('Sample shape:', sample_shape)
    print('Return sample tensor shape:', distribution.sample(sample_shape).shape)
    
def describe_sample_tensor_shapes(distributions, sample_shapes):
    started = False
    for distribution in distributions:
        print(distribution)
        for sample_shape in sample_shapes:
            describe_sample_tensor_shape(sample_shape, distribution)
        print()
        
sample_shapes = [1, 2, [1, 5], [3, 4, 5]]
describe_sample_tensor_shapes(poisson_distributions, sample_shapes)

tfp.distributions.Poisson("One Poisson Scalar Batch/", batch_shape=(), event_shape=(), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1,)
Sample shape: 2
Return sample tensor shape: (2,)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Return sample tensor shape: (3, 4, 5)

tfp.distributions.Poisson("Three Poissons/", batch_shape=(3,), event_shape=(), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1, 3)
Sample shape: 2
Return sample tensor shape: (2, 3)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Return sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Poisson("Two-by-three Poissons/", batch_shape=(2, 3), event_shape=(), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1, 2, 3)
Sample shape: 2
Return sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Return sample tensor shape: (3, 4, 5, 2, 3)

tfp.distributions.

In [6]:
describe_sample_tensor_shapes(normal_distribution, sample_shapes)

tfp.distributions.Normal("Standard/", batch_shape=(), event_shape=(), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1,)
Sample shape: 2
Return sample tensor shape: (2,)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Return sample tensor shape: (3, 4, 5)

tfp.distributions.Normal("Standard Vector Batch/", batch_shape=(1,), event_shape=(), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1, 1)
Sample shape: 2
Return sample tensor shape: (2, 1)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5, 1)
Sample shape: [3, 4, 5]
Return sample tensor shape: (3, 4, 5, 1)

tfp.distributions.Normal("Different Locs/", batch_shape=(4,), event_shape=(), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1, 4)
Sample shape: 2
Return sample tensor shape: (2, 4)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5, 4)
Sample shape: [3, 4, 5]
Return sample tensor shape: (3, 4, 5, 4)

tfp.distributions.Normal("Broadcasting Scale/", bat

#### Computing `log_prob` For Scalar Distributions
`log_prob` takes as input a (non-empty) tensor representing the location(s) at which to compute the `log_prob` for the distribution. In the most straightforward case, this tensor will have a shape of the form `[sample_shape, batch_shape, event_shape]`, where `batch_shape` and `event_shape` match the batch and event shapes of the distribution. Recall once more that for scalar distributions, `event_shape = []`, so the input tensor has shape `[sample_shape, batch_shape]` In this case, we get back a tensor of shape `[sample_shape, batch_shape]`:

In [7]:
three_poissons = tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons')
three_poissons

<tfp.distributions.Poisson 'Three Poissons/' batch_shape=(3,) event_shape=() dtype=float32>

In [8]:
three_poissons.log_prob([[1., 10., 100.], [100, 10, 1]]) # sample_shape is [2]

<tf.Tensor: id=666, shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -2.0785608,   -3.222351 ],
       [-364.73938  ,   -2.0785608,  -95.39483  ]], dtype=float32)>

In [9]:
three_poissons.log_prob([[[[1., 10., 100.], [100., 10., 1.]]]]) # sample shape is [1, 1, 2].

<tf.Tensor: id=683, shape=(1, 1, 2, 3), dtype=float32, numpy=
array([[[[  -1.       ,   -2.0785608,   -3.222351 ],
         [-364.73938  ,   -2.0785608,  -95.39483  ]]]], dtype=float32)>

Note how in the first example, the input and output have shape [2, 3] and in the second example they have shape [1, 1, 2, 3].

In [10]:
three_poissons.log_prob([10.])

<tf.Tensor: id=700, shape=(3,), dtype=float32, numpy=array([-16.104412 ,  -2.0785608, -69.052704 ], dtype=float32)>

The tensor `[10.]` (with shape [1]) is broadcast across the `batch_shape` of 3, so we evaluate all three Poissons' log probability at the value 10.

In [11]:
three_poissons.log_prob([[[1.], [10.]], [[100.], [1000.]]])

<tf.Tensor: id=717, shape=(2, 2, 3), dtype=float32, numpy=
array([[[-1.0000000e+00, -7.6974149e+00, -9.5394829e+01],
        [-1.6104412e+01, -2.0785608e+00, -6.9052704e+01]],

       [[-3.6473938e+02, -1.4348087e+02, -3.2223511e+00],
        [-5.9131284e+03, -3.6195432e+03, -1.4069580e+03]]], dtype=float32)>

In the above example, the input tensor has shape `[2, 2, 1]`, while the distributions object has a batch shape of 3. So for each of the `[2, 2]` sample dimensions, the single value provided gets broadcast to each of the three Poissons.

A possibly useful way to think of it: because `three_poissons` has `batch_shape = [2, 3]`, a call to `log_prob` must take a Tensor whose last dimension is either 1 or 3; anything else is an error. (The numpy broadcasting rules treat the special case of a scalar as being totally equivalent to a Tensor of shape `[1]`.)

In [12]:
poisson_2_by_3 = tfd.Poisson(
    rate=[[1., 10., 100.,], [2., 20., 200.]],
    name='Two-by-Three Poissons')

In [13]:
poisson_2_by_3.log_prob(1.)

<tf.Tensor: id=736, shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39483  ],
       [  -1.3068528,  -17.004269 , -194.70168  ]], dtype=float32)>

In [14]:
poisson_2_by_3.log_prob([1., 10., 100.])

<tf.Tensor: id=753, shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.222351 ],
       [ -1.3068528,  -5.14709  , -33.907654 ]], dtype=float32)>

In [16]:
poisson_2_by_3.log_prob([[1., 10., 100.], [1., 10., 100.]])

<tf.Tensor: id=770, shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.222351 ],
       [ -1.3068528,  -5.14709  , -33.907654 ]], dtype=float32)>

In [17]:
poisson_2_by_3.log_prob([[1., 1., 1.], [2., 2., 2.]])

<tf.Tensor: id=787, shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39483  ],
       [  -1.3068528,  -14.701683 , -190.09651  ]], dtype=float32)>

In [18]:
poisson_2_by_3.log_prob([[1.], [2.]])

<tf.Tensor: id=804, shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39483  ],
       [  -1.3068528,  -14.701683 , -190.09651  ]], dtype=float32)>

The above examples involved broadcasting over the batch, but the sample shape was empty. Suppose we have a collection of values, and we want to get the log probability of each value at each point in the batch. We could do it manually:

In [19]:
poisson_2_by_3.log_prob([[[1., 1., 1.], [1., 1., 1.]], [[2., 2., 2.], [2., 2., 2.]]])

<tf.Tensor: id=821, shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39483  ],
        [  -1.3068528,  -17.004269 , -194.70168  ]],

       [[  -1.6931472,   -6.087977 ,  -91.4828   ],
        [  -1.3068528,  -14.701683 , -190.09651  ]]], dtype=float32)>

Or we could let broadcasting handle the last batch dimension:

In [20]:
poisson_2_by_3.log_prob([[[1.], [1.]], [[2.], [2.]]])

<tf.Tensor: id=838, shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39483  ],
        [  -1.3068528,  -17.004269 , -194.70168  ]],

       [[  -1.6931472,   -6.087977 ,  -91.4828   ],
        [  -1.3068528,  -14.701683 , -190.09651  ]]], dtype=float32)>

Suppose we had a long list of values we wanted to evaluate at every batch point. For that, the following notation, which adds extra dimensions of size 1 to the right side of the shape, is extremely useful:

In [21]:
poisson_2_by_3.log_prob(tf.constant([1., 2.])[..., tf.newaxis, tf.newaxis])

<tf.Tensor: id=859, shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39483  ],
        [  -1.3068528,  -17.004269 , -194.70168  ]],

       [[  -1.6931472,   -6.087977 ,  -91.4828   ],
        [  -1.3068528,  -14.701683 , -190.09651  ]]], dtype=float32)>

This is an instance of strided slice notation.

In [22]:
three_poissons.log_prob([[1.], [10.], [50.], [100.]])

<tf.Tensor: id=876, shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39483  ],
       [ -16.104412 ,   -2.0785608,  -69.052704 ],
       [-149.47777  ,  -43.34851  ,  -18.219254 ],
       [-364.73938  , -143.48087  ,   -3.222351 ]], dtype=float32)>

In [23]:
three_poissons.log_prob(tf.constant([1., 10., 50., 100.])[..., tf.newaxis])

<tf.Tensor: id=897, shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39483  ],
       [ -16.104412 ,   -2.0785608,  -69.052704 ],
       [-149.47777  ,  -43.34851  ,  -18.219254 ],
       [-364.73938  , -143.48087  ,   -3.222351 ]], dtype=float32)>

### Multivariate distributions
We now turn to multivariate distributions, which have non-empty event shape. Let's look at multinomial distributions

In [26]:
multinomial_distributions = [
     # Multinomial is a vector-valued distribution: if we have k classes,
     # an individual sample from the distribution has k values in it, so the
     # event shape is `[k]`.
    tfd.Multinomial(total_count=100, probs=[.5, .4, .1],
                   name='One Multinomial'),
    tfd.Multinomial(total_count=[100., 1000.], probs=[.5,.4, .1],
                   name='Two Multinomials Same Probs'),
    tfd.Multinomial(total_count=100., probs=[[.5, .4, .1], [.1, .2, .7]],
                   name='Two Multinomials Same Counts'),
    tfd.Multinomial(total_count=[100., 1000],
                   probs=[[.5, .4, .1],[.1, .2, .7]],
                   name='Two Multinomials Different Everything')
]
describe_distributions(multinomial_distributions)

tfp.distributions.Multinomial("One Multinomial/", batch_shape=(), event_shape=(3,), dtype=float32)
tfp.distributions.Multinomial("Two Multinomials Same Probs/", batch_shape=(2,), event_shape=(3,), dtype=float32)
tfp.distributions.Multinomial("Two Multinomials Same Counts/", batch_shape=(2,), event_shape=(3,), dtype=float32)
tfp.distributions.Multinomial("Two Multinomials Different Everything/", batch_shape=(2,), event_shape=(3,), dtype=float32)


In [27]:
describe_sample_tensor_shapes(multinomial_distributions, sample_shapes)

tfp.distributions.Multinomial("One Multinomial/", batch_shape=(), event_shape=(3,), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1, 3)
Sample shape: 2
Return sample tensor shape: (2, 3)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Return sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Multinomial("Two Multinomials Same Probs/", batch_shape=(2,), event_shape=(3,), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1, 2, 3)
Sample shape: 2
Return sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Return sample tensor shape: (3, 4, 5, 2, 3)

tfp.distributions.Multinomial("Two Multinomials Same Counts/", batch_shape=(2,), event_shape=(3,), dtype=float32)
Sample shape: 1
Return sample tensor shape: (1, 2, 3)
Sample shape: 2
Return sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Return sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Return sample 

Computing log probabilities is equally straightforward. Let's work on an example with diagonal Multivariate Normal distributions. (Multinomials are not very broadcast friendly, since the constraints on the counts and probabilities mean broadcasting will often produce inadmissible values.) We'll use . batch of 2 3-D distributions with the same mean but different scales (standard deviations):

In [30]:
two_multivariate_normals = tfd.MultivariateNormalDiag(loc=[1., 2., 3.], scale_identity_multiplier=[1., 2.])
two_multivariate_normals

<tfp.distributions.MultivariateNormalDiag 'MultivariateNormalDiag/' batch_shape=(2,) event_shape=(3,) dtype=float32>

Now let's evaluate the log probability of each batch point at its mean and at a shifted mean:

In [31]:
two_multivariate_normals.log_prob([[[1., 2., 3.]], [[3., 4., 5]]])

<tf.Tensor: id=2804, shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

Exactly equivalently, we can use [Strided slide](https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/strided-slice) to insert an extra shape=1 dimension in the middle of a constant.

In [32]:
two_multivariate_normals.log_prob(
    tf.constant([[1., 2., 3.], [3., 4., 5.]])[:, tf.newaxis, :])

<tf.Tensor: id=2830, shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

On the other hand, if we don't insert the extra dimension, we pass `[1, 2, 3]` to the first batch point and `[3, 4, 5]` to the second:

In [33]:
two_multivariate_normals.log_prob(tf.constant([[1., 2., 3], [3, 4, 5]]))

<tf.Tensor: id=2852, shape=(2,), dtype=float32, numpy=array([-2.7568154, -6.336257 ], dtype=float32)>

## Shape Manipulation Techiques
### The Reshape Bijector
The `Reshape` bijector can be used to reshape the *event_shape* of a distribution. Let's see an example:

In [34]:
six_way_multinomial = tfd.Multinomial(total_count=1000., probs=[.3, .25, .2, .15, .08, .02])
six_way_multinomial

<tfp.distributions.Multinomial 'Multinomial/' batch_shape=() event_shape=(6,) dtype=float32>

We created a multinomial with an event shape of `[6]`. The Reshape Bijector allows us to treat this as a distribution with an event shape of `[2, 3]`.

A `Bijector` represents a differentiable, one-to-one function on an open subset of $\mathbb{R}^n$. `Bijectors` are used in conjunction with `TransformedDistribution`, which models a distribution $p(y)$ in terms of a base distribution $p(x)$ and a `Bijector` that represents $Y=g(X)$. 

In [35]:
transformed_multinomial = tfd.TransformedDistribution(
    distribution=six_way_multinomial,
    bijector=tfb.Reshape(event_shape_out=[2, 3]))
transformed_multinomial

<tfp.distributions.TransformedDistribution 'reshapeMultinomial/' batch_shape=() event_shape=(2, 3) dtype=float32>

In [36]:
six_way_multinomial.log_prob([500, 100, 100, 150, 100, 50])

<tf.Tensor: id=2940, shape=(), dtype=float32, numpy=-178.21973>

In [37]:
transformed_multinomial.log_prob([[500, 100, 100], [150, 100, 50]])

<tf.Tensor: id=3012, shape=(), dtype=float32, numpy=-178.21973>

This is the *only* thing the `Reshape` bijector can do: it cannot turn event dimensions into batch dimensions or vice-versa.

### The Independent Distribution
The `Independent` distribution is used to treat a collection of independent, not-necessarily-identical (aka a batch of) distributions as a single distribution. More concisely, `Independent` allows us to convert dimensions in `batch_shape` to dimensions in `event_shape`. We'll illustrate by example.

In [38]:
two_by_five_bernoulli = tfd.Bernoulli(
    probs=[[0.05, .1, .15, .2, .25], [.3, .35, .4, .45, .5]],
    name='Two By Five Bernoulli')
two_by_five_bernoulli

<tfp.distributions.Bernoulli 'Two By Five Bernoulli/' batch_shape=(2, 5) event_shape=() dtype=int32>

We can think of this as two-by-five array of coins with the associated probabilities of heads. Let's evaluate the probability of a particular, arbitrary set of ones and zeros:

In [39]:
pattern = [[1., 0., 0., 1., 0.], [0., 0., 1, 1, 1]]
two_by_five_bernoulli.log_prob(pattern)

<tf.Tensor: id=3031, shape=(2, 5), dtype=float32, numpy=
array([[-2.9957323 , -0.10536051, -0.16251893, -1.609438  , -0.28768206],
       [-0.35667494, -0.43078294, -0.9162907 , -0.79850775, -0.6931472 ]],
      dtype=float32)>

We can use `Independent` to turn this into two different "sets of five Bernoullis', which is useful if we want to consider a 'row' of coin flips coming up in a given pattern as a single outcome:

In [40]:
two_sets_of_five = tfd.Independent(distribution=two_by_five_bernoulli,
                                  reinterpreted_batch_ndims=1,
                                  name='Two sets of five')
two_sets_of_five

<tfp.distributions.Independent 'Two sets of five/' batch_shape=(2,) event_shape=(5,) dtype=int32>

Mathematically, we are computing the log probability of each 'set' of five by summing the log probabilities of the five 'independent' coin flips in the set, which is where the distribution gets its name:

In [41]:
two_sets_of_five.log_prob(pattern)

<tf.Tensor: id=3049, shape=(2,), dtype=float32, numpy=array([-5.160732 , -3.1954036], dtype=float32)>

We can go even further and use `Independent` to create a distribution where individual events are a set of two-by-five Bernoullis:

In [42]:
one_set_of_two_by_five = tfd.Independent(
    distribution=two_by_five_bernoulli, reinterpreted_batch_ndims=2,
    name="One set of two by five")
one_set_of_two_by_five

<tfp.distributions.Independent 'One set of two by five/' batch_shape=() event_shape=(2, 5) dtype=int32>

It's worth noting that from the perspective of `sample`, using `Independent` changes nothing:

In [43]:
describe_sample_tensor_shapes(
    [two_by_five_bernoulli,
    two_sets_of_five,
    one_set_of_two_by_five], ([[3, 5]]))

tfp.distributions.Bernoulli("Two By Five Bernoulli/", batch_shape=(2, 5), event_shape=(), dtype=int32)
Sample shape: [3, 5]
Return sample tensor shape: (3, 5, 2, 5)

tfp.distributions.Independent("Two sets of five/", batch_shape=(2,), event_shape=(5,), dtype=int32)
Sample shape: [3, 5]
Return sample tensor shape: (3, 5, 2, 5)

tfp.distributions.Independent("One set of two by five/", batch_shape=(), event_shape=(2, 5), dtype=int32)
Sample shape: [3, 5]
Return sample tensor shape: (3, 5, 2, 5)

